In [93]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

In [94]:
caffe.set_mode_cpu()
net = caffe.Net('../../models/swimmers_fullconv/swimmers_fullconv_big_51x51.prototxt', 
                '../../models/swimmers_fullconv/swimmers_fullconv_big_51x51.protobuf', caffe.TEST)
net.reshape()

In [95]:
# find max_kernels weight blob
max_kernels = net.params['idpr_dt'][0].data
# max_kernels.reshape(1, 152, 35, 35)
print max_kernels.shape
print max_kernels

(1, 152, 51, 51)
[[[[ 0.18185     0.17472148  0.16788392 ...,  0.16788392  0.17472148
     0.18185   ]
   [ 0.17472148  0.16759296  0.1607554  ...,  0.1607554   0.16759296
     0.17472148]
   [ 0.16788392  0.1607554   0.15391785 ...,  0.15391785  0.1607554
     0.16788392]
   ..., 
   [ 0.16788392  0.1607554   0.15391785 ...,  0.15391785  0.1607554
     0.16788392]
   [ 0.17472148  0.16759296  0.1607554  ...,  0.1607554   0.16759296
     0.17472148]
   [ 0.18185     0.17472148  0.16788392 ...,  0.16788392  0.17472148
     0.18185   ]]

  [[ 0.12996273  0.12486818  0.11998159 ...,  0.11998159  0.12486818
     0.12996273]
   [ 0.12486818  0.11977365  0.11488705 ...,  0.11488705  0.11977365
     0.12486818]
   [ 0.11998159  0.11488705  0.11000045 ...,  0.11000045  0.11488705
     0.11998159]
   ..., 
   [ 0.11998159  0.11488705  0.11000045 ...,  0.11000045  0.11488705
     0.11998159]
   [ 0.12486818  0.11977365  0.11488705 ...,  0.11488705  0.11977365
     0.12486818]
   [ 0.12996273  0.

In [96]:
# find scale term weight blob
scale_terms = net.params['idpr_dt'][1].data
print scale_terms.shape

(152,)


In [97]:
weightsfile = '/data/hdd/harzig/idpr/idpr_code/weights.txt'


def getDefWeight(fname, idx):

    with open(fname) as f:
        content = f.readlines()
        
    nums = content[0].split("\t")
    nbApp = int(nums[0])
    nbIdpr = int(nums[1])
    nbDef = int(nums[2])
    nbBias = int(nums[3])

    #print nbApp, nbIdpr, nbDef, nbBias
    arr = content[1].split("\t")[:-1]
    #print (len(arr))
    w = map(float, arr)
    return w[nbApp+nbIdpr+2*idx]

#for line in content:
#    splitted = line.split("\t")
#    from_ = int(splitted[0])
#    to_ = int(splitted[1])
#    cluster_ = int(splitted[2])
#    n_ = int(splitted[3])
#    classes[idx] = map(int, splitted[4:4+n_]) 
#    idx+=1
#    #print idx
    

In [81]:
getDefWeight(weightsfile, 0)

0.000145480002192

In [88]:

def get_kernel_for_idx(idx):
    multiplier = getDefWeight(weightsfile, idx)
    kernel = np.zeros((max_kernels.shape[2], max_kernels.shape[3]))
    half_size = (kernel.shape[0] - 1) /2
    tmp_sum = 0

    for x in xrange(-half_size, half_size+1):
        for y in xrange(-half_size, half_size+1):
            pixel = multiplier * (x**2 + y**2)
            tmp_sum += pixel
            kernel[x + half_size ,y + half_size] = pixel

    return kernel
    #print kernel #* (1 / tmp_sum)

In [89]:
# write every kernel as the same!
for i in xrange(max_kernels.shape[1]):
    max_kernels[0, i] = get_kernel_for_idx(i)

In [90]:
# write scale terms to 1 (for now...)

for i in xrange(scale_terms.shape[0]):
    scale_terms[i] = 1

In [91]:
print max_kernels, scale_terms

[[[[ 0.18185     0.17472148  0.16788392 ...,  0.16788392  0.17472148
     0.18185   ]
   [ 0.17472148  0.16759296  0.1607554  ...,  0.1607554   0.16759296
     0.17472148]
   [ 0.16788392  0.1607554   0.15391785 ...,  0.15391785  0.1607554
     0.16788392]
   ..., 
   [ 0.16788392  0.1607554   0.15391785 ...,  0.15391785  0.1607554
     0.16788392]
   [ 0.17472148  0.16759296  0.1607554  ...,  0.1607554   0.16759296
     0.17472148]
   [ 0.18185     0.17472148  0.16788392 ...,  0.16788392  0.17472148
     0.18185   ]]

  [[ 0.12996273  0.12486818  0.11998159 ...,  0.11998159  0.12486818
     0.12996273]
   [ 0.12486818  0.11977365  0.11488705 ...,  0.11488705  0.11977365
     0.12486818]
   [ 0.11998159  0.11488705  0.11000045 ...,  0.11000045  0.11488705
     0.11998159]
   ..., 
   [ 0.11998159  0.11488705  0.11000045 ...,  0.11000045  0.11488705
     0.11998159]
   [ 0.12486818  0.11977365  0.11488705 ...,  0.11488705  0.11977365
     0.12486818]
   [ 0.12996273  0.12486818  0.11998

In [92]:
# save changes to new binaryproto
net.save('../../models/swimmers_fullconv/swimmers_fullconv_big_51x51.protobuf')